In [88]:
import networkx as nx
import numpy as np
import keras as k
from keras.layers import Dense, Bidirectional, Embedding, LSTM
from keras import Model
import keras.optimizers as opt
import keras.losses as losses
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [89]:
# read Embedding Vocab
vocab_file = "w2v_vocab_blogcatalog.txt"
embedd_file_w2v = "w2v_blogcatalog.txt"
embedd_file_ft = "ft_blogcatalog.txt"

In [90]:
G = nx.read_gml("graphs/blogcatalog.gml")

In [91]:
x = []
Y = []
for node in G.nodes(data=True):
    x.append(node[0])
    if type(node[1]["labels"]) == int:
        Y.append([node[1]["labels"]])
    else:
        Y.append(node[1]["labels"])

In [92]:
with open(embedd_file_ft) as f:
    embedds = f.readlines()

In [93]:
'''with open(embedd_file_w2v) as f:
    embedds = f.readlines()'''

'with open(embedd_file_w2v) as f:\n    embedds = f.readlines()'

In [94]:
embeddings = {}

In [95]:
for e in embedds[1:]:
    embeddings[e.split(" ")[0]]=np.array(e.split(" ")[1:],dtype="float32")

In [96]:
X=[]
for _ in x:
    X.append(embeddings[_])
X=np.array(X)

In [97]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Y_ = mlb.fit_transform(Y)

In [98]:
split = 0.9
dsSize = X.shape[0]
train = int(dsSize*split)

X_train = X[:train]
Y_train = Y_[:train]

X_test = X[train:]
Y_test = Y_[train:]

In [99]:
lr = OneVsRestClassifier(LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'))
#lr = OneVsRestClassifier(LogisticRegression(random_state=0, solver='saga',multi_class='multinomial'))

In [100]:
ovr = lr.fit(X_train,Y_train)

In [101]:
from sklearn.metrics import classification_report

In [102]:
Y_hat = ovr.predict(X_test)

In [103]:
from sklearn.metrics import f1_score

In [104]:
f1_score(Y_test,Y_hat, average='micro')  

0.31492618917441223

In [105]:
f1_score(Y_test,Y_hat, average='macro')  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.17193511382883675

In [ ]:
#print(classification_report(Y_test,Y_hat))